Setup

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import sys
import json

from datetime import datetime
from itertools import combinations
from openai import OpenAI
from collections import defaultdict

In [ ]:
models = {
    'Gemma-V3-1B': 'google/gemma-3-1b-it:free',
    'DeepSeek-V3-0324': 'deepseek/deepseek-chat-v3-0324:free'
}

base_urls = {
    'OpenRouter': 'https://openrouter.ai/api/v1',
}

clients_details = [
    {
        'base_url': base_urls['OpenRouter'],
        'api_key': "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"
    }, # zandiyeh1379
    {
        'base_url': base_urls['OpenRouter'],
        'api_key': "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"
    }, # oibf7950
    {
        'base_url': base_urls['OpenRouter'],
        'api_key': "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"
    },  # moein
]

clients = [
    OpenAI(
        base_url=client['base_url'],
        api_key=client['api_key']
    ) for client in clients_details
]

In [ ]:
current_client_idx = 0
current_client = clients[current_client_idx]

def update_current_client():
  global current_client_idx, current_client
  current_client_idx = (current_client_idx + 1) % len(clients)
  current_client = clients[current_client_idx]

Social Network Creation

In [ ]:
class Agent:
  def __init__(self, persona: dict):
    self.persona = persona

  def get_details(self):
    return ', '.join(f"{k}: {v}" for k, v in self.persona.items())

In [ ]:
class SocialNetwork:
  def __init__(self, agents: list[Agent]):
    self.agents = agents
    self.network = self.__create_network()
    self.groups = []

  def __create_network(self):
    network = nx.Graph()
    network.add_nodes_from(range(len(self.agents)))
    return network

  def draw_network(self):
    nx.draw(self.network, with_labels=True)
    plt.show()

  def create_group(self, members: list[int], model: str):
    connections = list(combinations(members, 2))
    self.network.add_edges_from(connections)

    members = [self.agents[member] for member in members]
    group = Group(members, model)
    self.groups.append(group)

    return group

In [ ]:
class Group:
  def __init__(self, members: list[Agent], model: str):
    self.members = members
    self.model = model
    self.chats = [] # [{'sender': <Agent>, 'message': <str>}, ...]

  def add_chat(self, member: Agent, message: str):
    chat = {'sender': member, 'message': message}
    self.chats.append(chat)

  def get_members_details(self, indices: list[int]):
    return '\n'.join(self.members[i].get_details() for i in indices)

  def get_chats_history(self, last_n: int|None = None):
    num_all_chats = len(self.chats)
    last_n = num_all_chats if (last_n is None) else min(last_n, num_all_chats)

    chats_history = ''
    for chat in self.chats[-last_n:]:
      chats_history += f"sender: {chat['sender'].persona['Name']}\n"
      chats_history += f"message: {chat['message']}\n\n"

    return chats_history

  def get_list_messages(self, member: Agent):
    return [chat['message'] for chat in self.chats if chat['sender'] == member]

  def get_dict_chats(self):
    dict_chats = defaultdict(lambda: {'messages': []})
    for chat in self.chats:
      dict_chats[chat['sender'].persona['Name']]['messages'].append(chat['message'])

    return dict(dict_chats)

  def send_message(self, sender_idx: int, prompt_start: str, len_chat_memory: int|None = None):
    sender_details = self.get_members_details([sender_idx])

    receivers_indices = [j for j in range(len(self.members)) if j != sender_idx]
    receivers_details = self.get_members_details(receivers_indices)

    prompt_intro = f"You are {sender_details}, engaging in a friendly conversation with:\n{receivers_details}\n\n"
    prompt_intro += f"Below is the chat history so far:\n\n{self.get_chats_history(len_chat_memory)}" #1. history of message

    def get_client_response():
      try:
        return current_client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    'role': 'user',
                    'content': f"{prompt_intro}\n{prompt_start}"
                }
            ],
        )
      except:
        return None

    num_failed_tries = 0
    while True:
        response = get_client_response()
        if response is not None:
            break
        num_failed_tries += 1
        if num_failed_tries > len(clients):
            sys.exit('No response from any client')
        update_current_client()

    return response.choices[0].message.content.strip()

  def communicate(self, prompt_start: str, rounds: int, len_chat_memory: int|None = None):
    for _ in range(rounds):
      for sender_idx in range(len(self.members)):
        message = self.send_message(sender_idx, prompt_start, len_chat_memory)
        self.add_chat(self.members[sender_idx], message)
        print(self.get_chats_history(1))

Experiments

In [ ]:
persona_keys = ['Name', 'Gender', 'Age', 'Economic Status', 'Occupation']
#2. change number of agents
#3. change the perosans
personas = [
  ['Kayla', 'Female', 'Teen', 'Working Class', 'TikTok Influencer'],
  ['Morgan', 'Nonbinary', 'Adult', 'Upper-Middle', 'Corporate Lawyer'],
  ['Frank', 'Male', 'Elderly', 'Poor', 'Uber Driver'], #4- socioeconomic diversity
  ['Karen', 'Female', 'Middle-Aged', 'Middle Class', 'Politician (Controversial)'],
  ['Leo', 'Male', 'Young Adult', 'Lower Class', 'Activist (Environmental)']
]

agents = [Agent(dict(zip(persona_keys, persona))) for persona in personas]

In [ ]:
social_network = SocialNetwork(agents)

group_0_1_2 = social_network.create_group([0, 1, 2], models['DeepSeek-V3-0324']) #5. Group vs. 1-on-1 chats
#6. Initial source of message

In [ ]:
social_network.draw_network()

In [ ]:
prompt_start = "Generate only the next message in the conversation, using your natural personality and communication style. Assume this is an online chat on social media, not an in-person conversation. If the chat history is empty, begin the conversation yourself—do not simulate the other participant's message."
# 7- sentiment tone of initial message,
# 8- Message type

In [ ]:
group_0_1_2.communicate(prompt_start, 5) # 1. number of interactions per agent

In [ ]:
group_0_1_2.get_list_messages(group_0_1_2.members[0])

In [ ]:
with open('exp1.json', 'w') as f:
    json.dump(group_0_1_2.get_dict_chats(), f)